# O Problema da Mochila

O objetivo deste tutorial é demonstrar como formular e resolver um
problema de otimização simples em JuMP, introduzindo as notações para variáveis, restrições, objetivos.

## Pacotes necessários

Este tutorial requer os seguintes pacotes:

In [1]:
using JuMP
import HiGHS

## Formulação

O [problema da mochila](https://en.wikipedia.org/wiki/Knapsack_problem)
é um problema clássico de otimização: dado um conjunto de itens e uma mochila com
uma capacidade fixa, temos que escolher um subconjunto de itens que caiba na mochila
e que tenha o maior valor total possível.

O nome do problema sugere uma analogia com o ato de fazer as malas para uma viagem,
onde o limite de peso da bagagem é a capacidade, e o objetivo é empacotar a melhor
combinação de pertences.

Podemos formular o problema da mochila como um programa linear com variáveis inteiras:
$$
\begin{aligned}
\max \; & \sum_{i=1}^n c_i x_i      \\
\text{s.a} \; & \sum_{i=1}^n w_i x_i \le C, \\
        & x_i \in \{0,1\},\quad \forall i=1,\ldots,n,
\end{aligned}
$$
onde $C$ é a capacidade, e há uma escolha entre $n$ itens, com
o item $i$ tendo peso $w_i$ e lucro $c_i$.
A variável de decisão $x_i$ é igual a 1 se o item for escolhido e 0 se não for.

Esta formulação pode ser escrita de forma mais compacta como:
$$
\begin{aligned}
\max \; & c^\top x       \\
\text{s.a} \; & w^\top x \le C \\
        & x \text{ binária }.
\end{aligned}
$$

## Dados

Os dados para o problema consistem em
- dois vetores (um para os lucros e outro para os pesos)
- a capacidade da mochila.

Temos cinco objetos:

In [2]:
n = 5;

Para nosso exemplo, usamos uma capacidade de 10 unidades:

In [3]:
capacidade = 10.0;

e os dados de lucro e peso:

In [4]:
lucro = [5.0, 3.0, 2.0, 7.0, 4.0];
peso = [2.0, 8.0, 4.0, 2.0, 5.0];

## Formulação JuMP

Vamos começar construindo o modelo JuMP para o problema da mochila.

Primeiro, vamos criar um objeto `Model` para armazenar os elementos do modelo enquanto
construímos cada parte.

Também definimos o _solver_ que será chamado para resolver o modelo.

In [5]:
modelo = Model(HiGHS.Optimizer)

A JuMP Model
├ solver: HiGHS
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

Em seguida, precisamos das variáveis de decisão que representam quais itens são escolhidos:

In [6]:
# [1:n] indica que o vetor x tem tamanho n
# Bin indica que x será uma variável binária
@variable(modelo, x[1:n], Bin)

5-element Vector{VariableRef}:
 x[1]
 x[2]
 x[3]
 x[4]
 x[5]

Vamos construir a restrição que limita o peso total a ser menor ou igual à capacidade dada:

In [7]:
@constraint(modelo, sum(peso[i] * x[i] for i in 1:n) <= capacidade)

2 x[1] + 8 x[2] + 4 x[3] + 2 x[4] + 5 x[5] <= 10

Finalmente, o objetivo é maximizar o lucro total dos itens escolhidos:

In [8]:
@objective(modelo, Max, sum(lucro[i] * x[i] for i in 1:n))

5 x[1] + 3 x[2] + 2 x[3] + 7 x[4] + 4 x[5]

Vamos imprimir uma descrição legível do modelo e verificar se ele
está conforme o esperado:

In [9]:
print(modelo)

Agora podemos resolver o problema de otimização:

In [10]:
# Aqui, veremos as mensagens do solver
# Isto pode ser desativado com set_silent(modelo)
optimize!(modelo)

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [2e+00, 8e+00]
  Cost   [2e+00, 7e+00]
  Bound  [1e+00, 1e+00]
  RHS    [1e+01, 1e+01]
Presolving model
1 rows, 5 cols, 5 nonzeros  0s
1 rows, 4 cols, 4 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   1 rows
   4 cols (4 binary, 0 integer, 0 implied int., 0 continuous)
   4 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

 z       0       0         0   0.

E ver algumas informações mais gerais sobre a solução:

In [11]:
solution_summary(modelo)

* Solver : HiGHS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "kHighsModelStatusOptimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 1.60000e+01
  Objective bound    : 1.60000e+01
  Relative gap       : 0.00000e+00
  Dual objective value : NaN

* Work counters
  Solve time (sec)   : 3.87073e-02
  Simplex iterations : 1
  Barrier iterations : -1
  Node count         : 1


Os itens escolhidos são

In [12]:
itens_escolhidos = [i for i in 1:n if value(x[i]) > 0.5]

3-element Vector{Int64}:
 1
 4
 5

Note que não comparamos com $1$, nem com $0$, mas com $0.5$.

Em um problema de maior porte, o _solver_ poderia não retornar exatamente $0$ e $1$, mas valores próximos.

In [13]:
value.(x)

5-element Vector{Float64}:
  1.0
  0.0
 -0.0
  1.0
  1.0

## Escrevendo uma função

Após trabalhar interativamente, é uma boa ideia implementar o modelo em uma função.
Assim, podemos
1. garantir que o modelo receberá dados de entrada com tipos bem definidos;
2. realizar a validação dos dados;
3. certificar que o processo de solução ocorreu conforme o esperado;
4. retornar os resultados de forma organizada;
5. reutilizar o modelo sem repetir código.

In [14]:
function resolver_problema_mochila(;
    lucro::Vector{Float64},
    peso::Vector{Float64},
    capacidade::Float64,
)
    n = length(peso)
    # Os vetores de lucro e peso devem ter o mesmo comprimento.
    @assert length(lucro) == n
    
    modelo = Model(HiGHS.Optimizer)
    set_silent(modelo)
    @variable(modelo, x[1:n], Bin)
    @objective(modelo, Max, lucro' * x)
    @constraint(modelo, peso' * x <= capacidade)
    
    optimize!(modelo)
    @assert termination_status(modelo) == OPTIMAL
    @assert primal_status(modelo) == FEASIBLE_POINT
    
    println("O valor ótimo é ", objective_value(modelo))
    println("A solução é:")
    for i in 1:n
        print("  x[$i] = ", round(Int, value(x[i])))
        println(", c[$i] / w[$i] = ", lucro[i] / peso[i])
    end
    
    itens_escolhidos = [i for i in 1:n if value(x[i]) > 0.5]
    return itens_escolhidos
end

itens = resolver_problema_mochila(; lucro = lucro, peso = peso, capacidade = capacidade)

O valor ótimo é 16.0
A solução é:
  x[1] = 1, c[1] / w[1] = 2.5
  x[2] = 0, c[2] / w[2] = 0.375
  x[3] = 0, c[3] / w[3] = 0.5
  x[4] = 1, c[4] / w[4] = 3.5
  x[5] = 1, c[5] / w[5] = 0.8


3-element Vector{Int64}:
 1
 4
 5

Note que, neste exemplo, os itens escolhidos (1, 4, e 5) são os que têm o maior "lucro por quilo".

A mochila ficou ao todo com 9 quilos: abaixo da capacidade de 10 quilos, mas sem espaço para mais um item.

In [15]:
sum(peso[itens])

9.0

## Exercícios

* Use dados diferentes
  - O que acontece ao aumentar a capacidade?
  - Será que você consegue construir um exemplo onde um item de "alto lucro por quilo" não é escolhido (mas que caberia na mochila sozinho)?
* Em vez de criar uma variável binária com `Bin`, poderíamos ter usado
  `@variable(model, 0 <= x[1:n] <= 1, Int)`.
  Veja que esta formulação dá a mesma solução.
  - O que acontece se deixamos usar 2 de cada item?
  - E se não houver limite para o número de itens? (Dica: basta não dar um limite superior para as variáveis `x`.)